# Training Notebook Can be found here
[Training Notebook](https://www.kaggle.com/shubham219/using-imagenet-params)

In [ ]:
!pip install ../input/kerasapplication/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/imageclassifiers/image_classifiers-1.0.0-py3-none-any.whl
!pip install ../input/efficientnet/efficientnet-1.1.1-py3-none-any.whl

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import glob
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras
from classification_models.tfkeras import Classifiers
from efficientnet.tfkeras import EfficientNetB4


SEED = 42


In [ ]:
weight_path = '../input/model-ensembling-with-k-fold/fineTuned_v0.77.h5'
my_model = load_model(weight_path)

In [ ]:
test_images = glob.glob('../input/cassava-leaf-disease-classification/test_images/*.jpg')

df_test = pd.DataFrame(test_images, columns = ['path'])



def make_test_gen(batch_size=16):
    
    my_test_idg = ImageDataGenerator()
    
    test_gen= my_test_idg.flow_from_dataframe(dataframe=df_test,
                                              x_col="path",
                                              y_col=None,
                                              batch_size=batch_size,
                                              seed=42,
                                              shuffle=False,
                                              class_mode=None,
                                              target_size=(512,512))
    return test_gen

In [ ]:
pred_list = []

for i in range(1):
    
    test_gen = make_test_gen(batch_size = 16)
    pred_test = my_model.predict(test_gen,  verbose = True)
    pred_list.append(pred_test.tolist())
                     
pred_test = np.mean(pred_list, axis=0)
pred_test_labels = np.argmax(pred_test, axis = -1)
final_submission = df_test

final_submission['image_id'] = final_submission.path.str.split('/').str[-1]
final_submission['label'] = pred_test_labels

final_csv = final_submission[['image_id', 'label']]
final_csv.head()
final_csv.to_csv('submission.csv', index=False)

In [ ]:
final_csv.head()